<a href="https://colab.research.google.com/github/haein0224/Study_machinelearning/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/HandsonMachinelearnig_Ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 7.1
# 투표기반 분류기 (moons dataset)

# 데이터 생성 (moon data)
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples = 1000, noise = 0.15, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=1004)

# 사용할 다른 종류의 분류기 import
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 각 분류기 세팅
log_clf = LogisticRegression(solver='liblinear')
rnd_clf = RandomForestClassifier(n_estimators=10)
svm_clf = SVC(gamma='auto')
svm_clf2 = SVC(gamma='auto', probability=True)


# voting classifier에 넣어줌
voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting = 'hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(solver='liblinear')),
                             ('rf', RandomForestClassifier(n_estimators=10)),
                             ('svc', SVC(gamma='auto'))])

In [19]:
# soft 간접투표를 이용하는 경우
voting_clf2 = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf2)],
    voting = 'soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(solver='liblinear')),
                             ('rf', RandomForestClassifier(n_estimators=10)),
                             ('svc', SVC(gamma='auto'))])

In [20]:
# 각 분류기의 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf, voting_clf2) :
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

  # Voting classifier의 정확도가 더 높음!

LogisticRegression 0.865
RandomForestClassifier 0.985
SVC 0.97
VotingClassifier 0.97
VotingClassifier 0.97


In [23]:
# 7.2 배깅과 페이스팅
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier # 결정트리 한가지 알고리즘을 이용해 앙상블

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500, # 총 500개의 예측기를 생성해 앙상블 진행
    max_samples = 100, bootstrap=True, n_jobs = -1 ) # n_jobs : 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정함 (-1로 지정하는 경우 가용한 모든 코어를 사용)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

accuracy_score(y_test, y_pred)
# 굉장히 높은 정확도 보임

0.965